In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import Dataset

import pandas as pd
import numpy as np
from PIL import Image
import tqdm
from glob import glob

# Mingxing Tan, Quoc V. Le, EfficientNet: Rethinking Model Scaling for Convolutional Neural Networks. ICML 2019
from efficientnet_pytorch import EfficientNet

In [2]:
PATH = './model.pth'

def evaluate_acc(model, dataloader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for i, (imgs, lbls) in enumerate(dataloader):
            outs = model(imgs.to(device))
            _, temp_preds = torch.max(outs, 1)
            total += lbls.size(0)
            temp_preds = temp_preds.cpu()
            correct += (temp_preds == lbls).sum().item()
    return correct / total

class GTSRBTest(Dataset):
    def __init__(self):
        self.imgs = []
        self.lbls = []
        
        self.img_transforms = transforms.Compose([
            transforms.Resize((128, 128)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        
        for filename in sorted(glob("./Final_Test/Images/*.ppm")):
            img = Image.open(filename).convert("RGB")
            self.imgs.append(img)
        
        df = pd.read_csv("./Final_Test/final_test.csv")
        self.lbls = list(df["ClassId"])
    
    def __getitem__(self, index):
        img = self.img_transforms(self.imgs[index])
        lbl = self.lbls[index]
        return img, lbl
    
    def __len__(self):
        return len(self.imgs)

In [3]:
test_set = GTSRBTest()
print(len(test_set))
test_loader = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False)

12630


In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # model with randomly intialized weights
        self.effNet = EfficientNet.from_name('efficientnet-b2')
        self.MLP = nn.Sequential(
            nn.Linear(1000, 200),
            nn.ReLU(),
            nn.Linear(200, 43)
        )
            
    def forward(self, x):
        x = self.effNet(x)
        x = self.MLP(x)
        return x
    
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("yes")
else:
    device = torch.device('cpu')
    
model = Net()
model.to(device)
model = torch.load(PATH)
test_accuracy = evaluate_acc(model, test_loader)
print(test_accuracy)

yes


/opt/anaconda3/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/opt/anaconda3/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/opt/anaconda3/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'torch.nn.modules.activation.ReLU' has changed. you can retrieve the original source code by accessing the object's source att

0.9775930324623912
